# Initial tests 

In [ ]:
import numpy as np
from scipy.stats import dirichlet, multinomial

# Define the model parameters
K = 3  # Number of populations
L = 5  # Number of loci
J = 5  # Number of alleles per locus

# Define the prior distributions
alpha = np.ones(J)  # Dirichlet prior parameter for allele frequencies

# Define the data
X = np.array([[1, 2, 3, 4, 5],
             [1, 2, 3, 4, 5],
             [1, 2, 3, 4, 5]])  # Genotype data

# Initialize the MCMC chain
Z = np.zeros(X.shape[0], dtype=int)  # Population assignments
P = np.zeros((K, L, J))  # Allele frequencies

# Sample population assignments, as a initial starting point assuming a uniform distr
for j in range(X.shape[0]):
    Z[j] = np.random.multinomial(1, np.ones(K) / K).argmax()

indexes_dict = get_indexes_of_population(Z)

for pop in range(Z.shape[1]):
    pop_samples = X[indexes_dict[pop]]
    for locus in range(X.shape[1]):
        values,prior = get_vals_and_alpha_prior(pop_samples)
        # Sample allele freqs from dirichlet prior from loci
        # Save allele in nonhomogeneous list of allele freqs
            # Make homogeneous matrix adding zeroes to nonexistent alleles
            # Shape is: K, L, #highest number of alleles in any loci
        # Make complete matrix of nonhomo list of allele freqs,

# Implement Z assignation from new allele freq matrix

# Run the MCMC chain
for i in range(1000):

    ## Given de pop assignments modify the alpha prior adding



    # Sample allele frequencies
    for k in range(K):
        for l in range(L):
            P[k, l, :] = np.random.dirichlet(alpha)

    # Update allele frequencies based on population assignments
    for j in range(X.shape[0]):
        for l in range(L):
            print(Z[j], l, X[j, l])
            P[Z[j], l, X[j, l]] += 1

# Normalize allele frequencies
for k in range(K):
    for l in range(L):
        P[k, l, :] /= np.sum(P[k, l, :])

# Print the results
print("Population assignments:", Z)
print("Allele frequencies:", P)

In [160]:

import numpy as np
test_data = np.random.randint(0,100,(4,5,2))
test_data
print(test_data)
print(test_data[:,1,:])

[[[28 69]
  [10 38]
  [63 75]
  [26 40]
  [27 34]]

 [[48 35]
  [50 51]
  [59 46]
  [37 58]
  [86 86]]

 [[ 2 85]
  [59 55]
  [61 36]
  [71 65]
  [64 37]]

 [[24 31]
  [99 56]
  [56 98]
  [83 91]
  [93 32]]]
[[10 38]
 [50 51]
 [59 55]
 [99 56]]


# Alpha testing

In [236]:
import numpy as np
from scipy.stats import dirichlet, multinomial


## Define functions
def get_vals_and_alpha_prior(data_mat):
    summary = np.unique(data_mat, return_counts = True)
    vals = summary[0]
    prior = summary[1] + 1/len(vals) ## Definition of prior in original paper?, counts + some alpha that always sums to 1

    return vals,prior


def get_vals_and_alpha_prior_full_alleles(data_mat,data_general):
    summary_locus_all_samples = np.unique(data_general) 
    summary_locus_pop = np.unique(data_mat, return_counts = True)
    
    # Create new column for unique values to save the population counts
    to_concatenate = np.zeros(summary_locus_all_samples.shape[0])
    summary_locus_all_samples = np.column_stack((summary_locus_all_samples, to_concatenate))

    if summary_locus_pop[0].shape[0] > 0: # If population wasnt sampled, no alleles would be found, this tests if this array is not empty
        # Add to the previous column the appropiate number of allele counts per locus per population
        for allele in range(summary_locus_all_samples.shape[0]):
            if summary_locus_all_samples[allele,0] in summary_locus_pop[0]:
                pop_allele_index = np.where(summary_locus_pop[0] == summary_locus_all_samples[allele,0])

                summary_locus_all_samples[allele,1] = summary_locus_pop[1][pop_allele_index[0][0]]

    # Generate values and prior
    vals = summary_locus_all_samples[:,0]
    prior = summary_locus_all_samples[:,1] + 1/len(vals) ## Definition of prior in original paper?, counts + some alpha that always sums to 1

    return vals,prior

# Create dictionary to find the index (samples) assignated to a population
def get_indexes_of_population(arr):
    unique_values = np.unique(arr)
    indexes_dict = {}
    for value in unique_values:
        #print(np.where(arr == value))
        indexes_dict[value] = list(np.where(arr == value)[0])
    return indexes_dict

# Sample population assignments, as a initial starting point assuming a uniform distr
# Assure that all populations are sampled to not generate problems down the line. Just resample if any pop is missing
def initial_pop_assignment(K,Z):
    all_pops_sampled = False
    while not(all_pops_sampled):
        for j in range(X.shape[0]):
            Z[j] = np.random.multinomial(1, np.ones(K) / K).argmax()
        if not(False in np.isin([x for x in range(K)],Z)):
            all_pops_sampled = True
    return Z

# Sample allele freqs on a per pooulation and per loci basis
def sample_allele_freqs(X,K, indexes_dict):
    # List that will save the dictionary- allele frequencies of the loci per population
    freq_dict_per_pop = []
    for pop in range(K):
        print(pop)
        print(indexes_dict[pop])
        pop_samples = X[indexes_dict[pop]]
        # Define population dictionary
        pop_dict = {}
        # Iterate over each loci within the population
        for locus in range(X.shape[1]):
            values,prior = get_vals_and_alpha_prior_full_alleles(pop_samples[:,locus,:],X)
            
            # Sample allele freqs from dirichlet prior from loci
            sampled_allele_freqs_L = np.random.dirichlet(prior)

            # Save allele freqs in population dictionary
            allele_dict = {}
            for value, freq in zip(values, sampled_allele_freqs_L):
                allele_dict[value] = freq
            pop_dict[locus] = allele_dict

        # Save population dictionary in freq_dict_per_pop
        freq_dict_per_pop.append(pop_dict)
    return freq_dict_per_pop


# Generate population assignment probabilites from estimated allele frequencies. 
# Estimate the "probability" of the full genotype of an individual in a population and ponder 

def generate_pop_probabilities(X, K, freq_dict_per_pop):
    sample_probs = []
    for sample in range(X.shape[0]):
        likelihoods = []
        for pop in range(K):
            pop_dict = freq_dict_per_pop[pop]
            product = 1
            for locus in range(X.shape[1]):
                genotype = X[sample,locus]
                allele_0_freq = pop_dict[locus][genotype[0]]
                allele_1_freq = pop_dict[locus][genotype[1]]
                product *= allele_0_freq*allele_1_freq
            likelihoods.append(product)
        probs = np.array(likelihoods)/sum(likelihoods)
        sample_probs.append(probs)
    sample_probs = np.array(sample_probs)
    return sample_probs


# Assign population from population probabilities estimated before
def assign_pop_from_probs(Z, K, X, sample_probs):
    for sample in range(X.shape[0]):
        Z[j] = np.random.multinomial(1, sample_probs[sample]).argmax()
    return Z


# Define the model parameters
K = 3  # Number of populations
L = 5  # Number of loci

# Define the prior distributions
#alpha = np.ones(J)  # Dirichlet prior parameter for allele frequencies

# Define the data
X = np.array([[[1, 1],
  [3, 3],
  [4, 4],
  [7, 7],
  [9, 9]],
 [[1, 2],
  [3, 3],
  [4, 6],
  [7, 7],
  [9, 9]],
 [[2, 2],
  [3, 3],
  [4, 5],
  [8, 8],
  [9, 9]],
 [[2, 2],
  [3, 3],
  [4, 5],
  [8, 7],
  [9, 10]]])  # Genotype data

# Initialize the MCMC chain
Z = np.zeros(X.shape[0], dtype=int)  # Population assignments
#P = np.zeros((K, L, J))  # Allele frequencies

# Sample population assignments, as a initial starting point assuming a uniform distr
Z = initial_pop_assignment(K,Z)


indexes_dict = get_indexes_of_population(Z)

print(Z.shape,Z)
# Sample allele freqs on a per pooulation and per loci basis

freq_dict_per_pop=sample_allele_freqs(X,K, indexes_dict)# Sample allele freqs on a per pooulation and per loci basis

for pop in range(K):
    print(freq_dict_per_pop[pop])

# Implement Z assignation from new allele freq matrix

sample_probs = generate_pop_probabilities(X, K, freq_dict_per_pop)
Z = assign_pop_from_probs(Z, K, X, sample_probs)
Z

(4,) [1 2 0 2]
0
[2]
1
[0]
2
[1, 3]
{0: {1.0: 2.2549199497391548e-12, 2.0: 0.5498784079960031, 3.0: 0.03224026221398752, 4.0: 0.001350643652609562, 5.0: 0.000964228391025687, 6.0: 0.0018052828575614085, 7.0: 5.853067496582071e-07, 8.0: 0.15722290896881433, 9.0: 8.594073921151104e-07, 10.0: 0.2565368212036017}, 1: {1.0: 0.032932394783837554, 2.0: 1.4797749665306442e-12, 3.0: 0.9543320850323141, 4.0: 1.281436309562029e-05, 5.0: 3.2226849262518485e-06, 6.0: 0.0011357444030866293, 7.0: 0.0002869330189201512, 8.0: 2.962956658915021e-06, 9.0: 3.0900326109308247e-13, 10.0: 0.011293842755372027}, 2: {1.0: 8.469512897005319e-08, 2.0: 2.790096688519279e-06, 3.0: 0.0033300703207478125, 4.0: 0.2410100727729683, 5.0: 0.48714145424586663, 6.0: 9.403139477031956e-06, 7.0: 0.031208395437879122, 8.0: 1.8307119626453428e-06, 9.0: 2.470645519083048e-07, 10.0: 0.23729565151472928}, 3: {1.0: 0.02879983095792928, 2.0: 2.1250010985197525e-05, 3.0: 7.567784330673003e-08, 4.0: 0.035920148837891436, 5.0: 2.1790

array([1, 2, 0, 2])

# Beta testing

In [ ]:
import numpy as np
from scipy.stats import dirichlet, multinomial


## Define functions

def get_vals_and_alpha_prior_full_alleles(data_mat,data_general):
    summary_locus_all_samples = np.unique(data_general) 
    summary_locus_pop = np.unique(data_mat, return_counts = True)
    
    # Create new column for unique values to save the population counts
    to_concatenate = np.zeros(summary_locus_all_samples.shape[0])
    summary_locus_all_samples = np.column_stack((summary_locus_all_samples, to_concatenate))

    if summary_locus_pop[0].shape[0] > 0: # If population wasnt sampled, no alleles would be found, this tests if this array is not empty
        # Add to the previous column the appropiate number of allele counts per locus per population
        for allele in range(summary_locus_all_samples.shape[0]):
            if summary_locus_all_samples[allele,0] in summary_locus_pop[0]:
                pop_allele_index = np.where(summary_locus_pop[0] == summary_locus_all_samples[allele,0])

                summary_locus_all_samples[allele,1] = summary_locus_pop[1][pop_allele_index[0][0]]

    # Generate values and prior
    vals = summary_locus_all_samples[:,0]
    prior = summary_locus_all_samples[:,1] + 1/len(vals) ## Definition of prior in original paper?, counts + some alpha that always sums to 1

    return vals,prior

# Create dictionary to find the index (samples) assignated to a population
def get_indexes_of_population(Z,K):
    unique_values = np.unique(Z)
    indexes_dict = {}
    for pop in range(K):
        #print(np.where(arr == value))
        indexes_dict[pop] = list(np.where(Z == pop)[0])
    return indexes_dict

# Sample population assignments, as a initial starting point assuming a uniform distr
# Assure that all populations are sampled to not generate problems down the line. Just resample if any pop is missing
def initial_pop_assignment(K,Z):
    all_pops_sampled = False
    while not(all_pops_sampled):
        for j in range(X.shape[0]):
            Z[j] = np.random.multinomial(1, np.ones(K) / K).argmax()
        if not(False in np.isin([x for x in range(K)],Z)):
            all_pops_sampled = True
    return Z

# Sample allele freqs on a per pooulation and per loci basis
def sample_allele_freqs(X,K, indexes_dict):
    # List that will save the dictionary- allele frequencies of the loci per population
    freq_dict_per_pop = []
    for pop in range(K):
        #print(pop)
        #print(indexes_dict[pop])
        pop_samples = X[indexes_dict[pop]]
        # Define population dictionary
        pop_dict = {}
        # Iterate over each loci within the population
        for locus in range(X.shape[1]):
            values,prior = get_vals_and_alpha_prior_full_alleles(pop_samples[:,locus,:],X)
            
            # Sample allele freqs from dirichlet prior from loci
            sampled_allele_freqs_L = np.random.dirichlet(prior)

            # Save allele freqs in population dictionary
            allele_dict = {}
            for value, freq in zip(values, sampled_allele_freqs_L):
                allele_dict[value] = freq
            pop_dict[locus] = allele_dict

        # Save population dictionary in freq_dict_per_pop
        freq_dict_per_pop.append(pop_dict)
    return freq_dict_per_pop


# Generate population assignment probabilites from estimated allele frequencies. 
# Estimate the "probability" of the full genotype of an individual in a population and ponder 

def generate_pop_probabilities(X, K, freq_dict_per_pop):
    sample_probs = []
    for sample in range(X.shape[0]):
        likelihoods = []
        for pop in range(K):
            pop_dict = freq_dict_per_pop[pop]
            product = 1
            for locus in range(X.shape[1]):
                genotype = X[sample,locus]
                allele_0_freq = pop_dict[locus][genotype[0]]
                allele_1_freq = pop_dict[locus][genotype[1]]
                product *= allele_0_freq*allele_1_freq
            likelihoods.append(product)
        probs = np.array(likelihoods)/sum(likelihoods)
        print("-",np.array(likelihoods),sum(likelihoods),probs)
        sample_probs.append(probs)
    sample_probs = np.array(sample_probs)
    return sample_probs


# Assign population from population probabilities estimated before
def assign_pop_from_probs(Z, K, X, sample_probs):
    for sample in range(X.shape[0]):
        Z[sample] = np.random.multinomial(1, sample_probs[sample]).argmax()
    return Z


#------------------------------------------------------------------------------------------

# Define the model parameters
K = 3  # Number of populations
L = 5  # Number of loci

# Define the data
X = np.array([[[1, 1],
  [3, 3],
  [4, 4],
  [7, 7],
  [9, 9]],
 [[1, 2],
  [3, 3],
  [4, 6],
  [7, 7],
  [9, 9]],
 [[2, 2],
  [3, 3],
  [4, 5],
  [8, 8],
  [9, 9]],
 [[2, 2],
  [3, 3],
  [4, 5],
  [8, 7],
  [9, 10]]])  # Genotype data

# Initialize the MCMC chain
iterations = 1000

Z_accum = []
P_accum = []

Z = np.zeros(X.shape[0], dtype=int)  # Population assignments
# Sample population assignments, as a initial starting point assuming a uniform distr
Z = initial_pop_assignment(K,Z)
Z_accum.append(Z)

# Run the MCMC chain

for i in range(iterations):
    # Get index from population

    indexes_dict = get_indexes_of_population(Z,K)
    print("Indexes", indexes_dict)

    # Sample allele freqs on a per pooulation and per loci basis

    freq_dict_per_pop = sample_allele_freqs(X,K, indexes_dict)# Sample allele freqs on a per pooulation and per loci basis
    P_accum.append(freq_dict_per_pop)

    # Implement Z assignation from new allele freq matrix

    sample_probs = generate_pop_probabilities(X, K, freq_dict_per_pop)
    #print(sample_probs)
    Z = assign_pop_from_probs(Z, K, X, sample_probs)
    
    print(Z)
    for freqs in freq_dict_per_pop:
        print(freqs)
    print("--------------------------------------------------------------")

print(Z_accum)
print(P_accum)

# Reading file implementation

# Random testing

In [206]:
def get_indexes_of_population_test(Z,K):
    unique_values = np.unique(Z)
    indexes_dict = {}
    for pop in range(K):
        #print(np.where(arr == value))
        indexes_dict[pop] = list(np.where(Z == pop)[0])
    return indexes_dict

test_pop_ass = np.array([1,1,1,0])
pops = 3
get_indexes_of_population_test(test_pop_ass,pops)

{0: [3], 1: [0, 1, 2], 2: []}

In [225]:

np.unique(X[[]][:,4,:], return_counts = True)[0].shape[0]


0

In [98]:
test_array = np.array([0,1,1,0])

np.isin([0,1,2],test_array)

array([ True,  True, False])

In [73]:
[x for x in range(K)]

[0, 1, 2]

In [148]:
def get_vals_and_alpha_prior_full_alleles(data_mat,data_general):
    summary_locus_all_samples = np.unique(data_general) 
    summary_locus_pop = np.unique(data_mat, return_counts = True)
    
    # Create new column for unique values to save the population counts
    to_concatenate = np.zeros(summary_locus_all_samples.shape[0])
    summary_locus_all_samples = np.column_stack((summary_locus_all_samples, to_concatenate))

    # Add to the previous column the appropiate number of allele counts per locus per population
    for allele in range(summary_locus_all_samples.shape[0]):
        if summary_locus_all_samples[allele,0] in summary_locus_pop[0]:
            pop_allele_index = np.where(summary_locus_pop[0] == summary_locus_all_samples[allele,0])

            summary_locus_all_samples[allele,1] = summary_locus_pop[1][pop_allele_index[0][0]]

    # Generate values and prior
    vals = summary_locus_all_samples[:,0]
    prior = summary_locus_all_samples[:,1] + 1/len(vals) ## Definition of prior in original paper?, counts + some alpha that always sums to 1

    return vals,prior

test_sample = np.arange(10)
test_pop = np.array([1,2,8,9])

print(test_sample)
print(test_pop)

get_vals_and_alpha_prior_test(test_pop,test_sample)

[0 1 2 3 4 5 6 7 8 9]
[1 2 8 9]
[[0. 0.]
 [1. 1.]
 [2. 1.]
 [3. 0.]
 [4. 0.]
 [5. 0.]
 [6. 0.]
 [7. 0.]
 [8. 1.]
 [9. 1.]]
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


(array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]),
 array([0.1, 1.1, 1.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1.1, 1.1]))

In [131]:

np.where(test_pop == 8)

(array([2]),)

In [198]:
np.array([1,1])/sum([1,1])

array([0.5, 0.5])

In [201]:
np.array([[0,1],[2,3],[4,5]])[0]


array([0, 1])

In [202]:
np.ones(K) / K

array([0.33333333, 0.33333333, 0.33333333])